In [13]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random
import seaborn as sns


import sqlite3
con = sqlite3.connect('db') 
def select(sql):
  return pd.read_sql(sql, con)

### **Task 1**


In [86]:
data_finished_lesson_test = {
    "date_created": pd.to_datetime([
        "2020-04-01 10:00:00", "2020-04-01 10:00:02", "2020-04-01 10:01:00",
        "2020-04-02 11:00:00", "2020-04-02 11:00:03", "2020-04-03 12:00:00",
        "2020-04-01 10:05:00", "2020-04-01 10:05:04"
    ]),
    "id": [1, 2, 3, 4, 5, 6, 7, 8],
    "lesson_id": [101, 102, 103, 101, 102, 103, 104, 105],
    "user_id": [1, 1, 1, 2, 2, 2, 1, 1]
}
finished_lesson_test = pd.DataFrame(data_finished_lesson_test)

data_lesson_index_test = {
    "lesson_id": [101, 102, 103, 104, 105],
    "lesson_name": ["Intro to Data", "Data Methods", "Data Analysis", "Advanced Topics", "Data Presentation"],
    "profession_id": [10, 10, 10, 10, 10],
    "profession_name": ["data-analyst", "data-analyst", "data-analyst", "data-analyst", "data-analyst"]
}
lesson_index_test = pd.DataFrame(data_lesson_index_test)

In [87]:
finished_lesson_test

,date_created,id,lesson_id,user_id
0,2020-04-01 10:00:00,1,101,1
1,2020-04-01 10:00:02,2,102,1
2,2020-04-01 10:01:00,3,103,1
3,2020-04-02 11:00:00,4,101,2
4,2020-04-02 11:00:03,5,102,2
5,2020-04-03 12:00:00,6,103,2
6,2020-04-01 10:05:00,7,104,1
7,2020-04-01 10:05:04,8,105,1


In [88]:
lesson_index_test

,lesson_id,lesson_name,profession_id,profession_name
0,101,Intro to Data,10,data-analyst
1,102,Data Methods,10,data-analyst
2,103,Data Analysis,10,data-analyst
3,104,Advanced Topics,10,data-analyst
4,105,Data Presentation,10,data-analyst


In [89]:
finished_lesson_test.to_sql('finished_lesson_test', con, index=False, if_exists='replace')
lesson_index_test.to_sql('lesson_index_test', con, index=False, if_exists='replace')

5

In [90]:
### Solution Task 1

In [91]:
sql = """
    SELECT a.user_id
        , li1.lesson_id
        , a.date_created AS lesson_datetime
        , b.date_created AS next_lesson_datetime
        , (julianday(b.date_created) - julianday(a.date_created)) * 86400 AS delta_seconds
        , li1.profession_name
        , li1.lesson_name AS current_lesson
        , li2.lesson_name AS next_lesson
    FROM finished_lesson_test a
    JOIN finished_lesson_test b ON a.user_id = b.user_id AND a.id < b.id
    JOIN lesson_index_test li1 ON a.lesson_id = li1.lesson_id
    JOIN lesson_index_test li2 ON b.lesson_id = li2.lesson_id
    WHERE li1.profession_name = li2.profession_name
      AND (julianday(b.date_created) - julianday(a.date_created)) * 86400 < 5
    ORDER BY a.user_id, a.date_created;
"""

select(sql)

,user_id,lesson_id,lesson_datetime,next_lesson_datetime,delta_seconds,profession_name,current_lesson,next_lesson
0,1,101,2020-04-01 10:00:00,2020-04-01 10:00:02,2.000029,data-analyst,Intro to Data,Data Methods
1,1,104,2020-04-01 10:05:00,2020-04-01 10:05:04,3.999978,data-analyst,Advanced Topics,Data Presentation
2,2,101,2020-04-02 11:00:00,2020-04-02 11:00:03,2.999984,data-analyst,Intro to Data,Data Methods
